# Extracting Observation Dense Bergs from Spreads

In [1]:
from icedef import iceberg

In [2]:
iip_season = 2015  # must be 2015 (more to come later)
iip_indices = range(3284, 3286)  # must be of type list or range

In [16]:
iip_df = iceberg.add_datetime_column(iceberg.get_iip_df(iip_season))

In [17]:
iip_df.sort_values('ICEBERG_NUMBER').head()

,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,TIMESTAMP
0,2015,1,11/21/2014,1639,59.98,-63.44,VIS,GEN,GEN,GCFR,2014-11-21 16:39:00
1,2015,1,12/13/2014,2138,58.32,-61.18,SAT-HIGH,RAD,RAD,RSA2,2014-12-13 21:38:00
2,2015,2,11/29/2014,2147,58.57,-60.53,SAT-HIGH,RAD,RAD,RSA2,2014-11-29 21:47:00
3,2015,2,12/6/2014,2142,58.21,-60.05,SAT-HIGH,RAD,RAD,RSA2,2014-12-06 21:42:00
4,2015,2,12/16/2014,1553,57.69,-59.16,VIS,LG,GEN,GTJZ,2014-12-16 15:53:00


In [18]:
tmp_df = iip_df['ICEBERG_NUMBER'].value_counts().rename('obs_counts').to_frame()

In [19]:
test_df = iip_df.merge(tmp_df, left_on='ICEBERG_NUMBER', right_index=True)

In [20]:
test_df = test_df.loc[test_df['obs_counts'] > 2]

In [21]:
test_df.head()

,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,TIMESTAMP,obs_counts
2,2015,2,11/29/2014,2147,58.57,-60.53,SAT-HIGH,RAD,RAD,RSA2,2014-11-29 21:47:00,3
3,2015,2,12/6/2014,2142,58.21,-60.05,SAT-HIGH,RAD,RAD,RSA2,2014-12-06 21:42:00,3
4,2015,2,12/16/2014,1553,57.69,-59.16,VIS,LG,GEN,GTJZ,2014-12-16 15:53:00,3
15,2015,11,1/6/2015,1842,51.53,-54.83,VIS,LG,DD,GTJZ,2015-01-06 18:42:00,5
16,2015,11,1/10/2015,1911,50.61,-54.78,R/V,GEN,GEN,GTJZ,2015-01-10 19:11:00,5


In [22]:
test_df['ICEBERG_NUMBER'].nunique()

1554

In [86]:
len(test_df['ICEBERG_NUMBER'].tolist())# == len(test_df)

8894

In [134]:
#%%timeit
berg_num = test_df['ICEBERG_NUMBER'].tolist()[0]
good_indices = []

berg_nums = test_df['ICEBERG_NUMBER'].tolist()
berg_times = test_df['TIMESTAMP'].tolist()

for i, row in test_df[:-2].iterrows():
    
    if i+1 >= len(test_df):
        break
        
    berg_num0 = berg_nums[i]
    berg_num1 = berg_nums[i+1]

    if berg_num0 == berg_num1:

        time0 = berg_times[i]
        time1 = berg_times[i+1]
        dt = time1 - time0
        dt_hours = dt.days*24 + dt.seconds/3600
        
        if dt_hours < 12:
            good_indices.append(i)
            good_indices.append(i+1)

good_indices = sorted(list(set(good_indices)))

In [135]:
len(test_df[test_df.index.isin(good_indices)])

647

In [141]:
test_df2 = test_df[test_df.index.isin(good_indices)]

In [142]:
tmp_df2 = test_df2['ICEBERG_NUMBER'].value_counts().rename('obs_counts').to_frame()

In [143]:
test_df3 = test_df2.merge(tmp_df2, left_on='ICEBERG_NUMBER', right_index=True)

In [147]:
test_df3.sort_values('obs_counts_y',ascending=False)

,ICEBERG_YEAR,ICEBERG_NUMBER,SIGHTING_DATE,SIGHTING_TIME,SIGHTING_LATITUDE,SIGHTING_LONGITUDE,SIGHTING_METHOD,SIZE,SHAPE,SOURCE,TIMESTAMP,obs_counts_x,obs_counts_y
5116,2015,21186,6/24/2015,1314,47.65,-47.07,R/V,MED,GEN,GMRS,2015-06-24 13:14:00,19,13
5107,2015,21186,5/16/2015,1355,47.84,-52.29,R/V,GEN,GEN,GMRS,2015-05-16 13:55:00,19,13
5115,2015,21186,6/21/2015,1645,47.83,-47.56,R/V,SM,DD,SHIP,2015-06-21 16:45:00,19,13
5114,2015,21186,6/19/2015,1653,47.77,-47.74,R/V,MED,DD,GPGR,2015-06-19 16:53:00,19,13
5113,2015,21186,6/19/2015,1111,47.83,-47.82,R/V,MED,TAB,GPGR,2015-06-19 11:11:00,19,13
5112,2015,21186,6/18/2015,1153,47.91,-48.31,VIS,MED,GEN,GTJZ,2015-06-18 11:53:00,19,13
5109,2015,21186,5/30/2015,1015,48.56,-51.87,R/V,SM,GEN,GTJZ,2015-05-30 10:15:00,19,13
5108,2015,21186,5/25/2015,1055,48.15,-52.09,R/V,SM,GEN,GPGR,2015-05-25 10:55:00,19,13
5118,2015,21186,7/14/2015,1346,46.20,-45.60,RAD,GEN,GEN,2006,2015-07-14 13:46:00,19,13
5106,2015,21186,5/15/2015,1243,47.82,-52.28,R/V,MED,TAB,GMRS,2015-05-15 12:43:00,19,13
